In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# import os
# from kaggle_secrets import UserSecretsClient
#
# GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [2]:
from IPython.display import Image, display, Markdown
from typing import Annotated, List, Literal, Optional, TypedDict

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.messages import AIMessage, BaseMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_core.tools import tool

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [3]:
# GLOBAL VARIABLES

# Names
bot_name = "Frenchie"
user_name = "You"

recursion_limit = 100

# Debug mode
debug = True

# Gemini model
llm_model = "gemini-2.0-flash"

base_llm = ChatGoogleGenerativeAI(
    model=llm_model, 
    temperature=0.5, 
    max_tokens=2000, 
    google_api_key=GOOGLE_API_KEY
    )

# Update learning history
history_learned_vocab = []

In [4]:
class UserState(TypedDict):
    """State representing the user's study conversation."""

    # The chat conversation.
    messages: Annotated[List[BaseMessage], add_messages]

    # The user's French CEFR level - A1, A2, B1, B2, C1, C2
    level: Optional[str]

    # The user's learned vocabulary -> For scalability, this can be changed to "topics_covered" to avoid saving a massive list of vocabulary
    learned_vocab: List[str]

    # The user's mode
    mode: Optional[str]

    # Whether the user has been greeted
    greeted: bool

    # Route to
    route_to: str

    # Stage
    stage: str

In [5]:
@tool
def get_user_level() -> str:
    """Get the user's CEFR level.
    Returns:
        The user's CEFR level saved currently, or None if no memory was saved.
    """

@tool
def get_learned_vocabulary() -> List[str]:
    """Get the user's learned vocabulary.
    Returns:
        The user's learned vocabulary saved currently, or None if no memory was saved.
    """

@tool
def get_learning_mode() -> str:
    """Get the user's learning mode.
    Returns:
        The user's learning mode saved currently, or None if no memory was saved.
    """

@tool
def set_user_level(level: str) -> str:
    """Set the user's CEFR level as "A1", "A2", "B1", "B2", "C1", or "C2" according to user's response.
    Returns:
        The updated CEFR level of the user, and begin the content generation stage.
    """

@tool
def update_learned_vocabulary(words: List[str]) -> List[str]:
    """Update the user's learned vocabulary with a list of words.
    Args:
        words: A list of words to add to the user's learned vocabulary.
    Returns:
        The updated learned vocabulary of the user.
    """

@tool
def set_learning_mode(mode: str) -> str:
    """Set the user's learning mode as "passive" or "active" according to user's response.
    Returns:
        The updated learning mode of the user.
    """

@tool
def exit_session() -> str:
    """The user is trying to exit the session. Get the learned_vocab list to print the necessary code block.
    Returns:
        The Python code block to initialize learned_vocab list.
    """

@tool
def update_stage(stage: str) -> str:
    """Update the user's stage in the conversation. They can be 'INIT_LEVEL', 'CONTENT', or 'EXIT'.
    Args:
        stage: The new stage to set for the user.
    Returns:
        The updated stage of the user.
    """


In [6]:
def process_tool_calls(state: UserState) -> UserState:
    if debug:
        print(f"------ NODE: Processing tool calls")
    """The user state node. This is where the user state is manipulated."""
    tool_msg = state.get("messages", [])[-1]
    learned_vocab = state.get("learned_vocab", [])
    mode = state.get("mode", None)
    level = state.get("level", None)
    route_to = state.get("route_to", None)
    stage = state.get("stage", None)
    outbound_msgs = []

    for tool_call in tool_msg.tool_calls:

        if tool_call["name"] == "get_user_level":
            if level is not None:
                response = f"The user's saved CEFR level is {level}."
            else:
                response = "No CEFR level saved yet."
        
        elif tool_call["name"] == "update_stage":
            if tool_call["args"] and isinstance(tool_call["args"]["stage"], str):
                stage = tool_call["args"]["stage"]
                state["stage"] = stage
                response = f"The user's stage has been updated to {stage}."
            else:
                response = "Invalid arguments for update_stage."
        
        elif tool_call["name"] == "get_learned_vocabulary":
            if learned_vocab:
                response = f"The user's learned vocabulary is: {', '.join(learned_vocab)}."
            else:
                response = "No learned vocabulary saved yet."
        
        elif tool_call["name"] == "get_learning_mode":
            if mode is not None:
                response = f"The user's saved learning mode is {mode}."
            else:
                response = "No learning mode saved yet."
        
        elif tool_call["name"] == "set_user_level":
            if tool_call["args"] and isinstance(tool_call["args"]["level"], str):
                level = tool_call["args"]["level"]
                state["level"] = level
                response = f"The user's CEFR level has been set to {level}. If no learning mode is selected yet, ask the user to choose one."
                stage = "CONTENT"
                state["stage"] = stage #move to content generation stage once level is set
            else:
                response = "Invalid arguments for set_user_level."
        
        elif tool_call["name"] == "update_learned_vocabulary":
            if tool_call["args"] and isinstance(tool_call["args"]["words"], list):
                learned_vocab.extend(tool_call["args"]["words"])
                learned_vocab = list(set(learned_vocab))
                state["learned_vocab"] = learned_vocab
                response = f"The user's learned vocabulary is updated to: {', '.join(learned_vocab)}."
            else:
                response = "Invalid arguments for update_learned_vocabulary."
        
        elif tool_call["name"] == "set_learning_mode":
            if tool_call["args"] and isinstance(tool_call["args"]["mode"], str):
                mode = tool_call["args"]["mode"]
                state["mode"] = mode
                response = f"The user's learning mode is set to {mode}."
            else:
                response = "Invalid arguments for set_learning_mode."
        
        elif tool_call["name"] == "exit_session":
            stage = "EXIT"
            state["stage"] = stage
            # route_to = "end_node"
            # state["route_to"] = route_to
            session_learned_vocab = state.get("learned_vocab", [])
            response = f"The user is trying to exit. The learned vocabulary is: {', '.join(session_learned_vocab)}. Return the code block to save the learned vocabulary."

        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')
        
        # Record the tool results as tool messages.
        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": outbound_msgs, "learned_vocab": learned_vocab, "mode": mode, "level": level, "route_to": route_to, "stage": stage}


In [7]:
tools = [get_user_level, get_learned_vocabulary, get_learning_mode, set_user_level, update_learned_vocabulary, set_learning_mode]
llm = base_llm.bind_tools(tools)

In [8]:
# The system instruction defines how the chatbot is expected to behave and includes
# rules for when to call different functions, as well as rules for the conversation, such
# as tone and what is permitted for discussion.
SYSINT = (
    "system",  # 'system' indicates the message is a system instruction.
    f"""You are {bot_name}, an interactive French vocabulary learning assistant for an English-speaking user. Your primary goal is to help users expand their French vocabulary based on their specified language level and learning preferences.

Say hello if it is the first interaction. You will then interact with the user to understand their language proficiency and learning demand. Once you've understood the user and their study mode, you will then generate relevant learning or testing content and guide them through the session.
You have access to tools to:
- Get and update the user's French CEFR level (`get_user_level`, `set_user_level`).
- Get and update the user's learned vocabulary (`get_learned_vocabulary`, `update_learned_vocabulary`).
- Get and set the current learning mode (`get_learning_mode`, `set_learning_mode`).
- Exit the session (`exit_session`).
- Update the user's stage in the conversation (`update_stage`). You only need to update the stage to "EXIT" when the user is trying to exit the session.


Remember to only discuss French vocabulary learning and stay strictly in the current step. Avoid making any assumptions about the user's intent at all costs. If a requested tool is unavailable, inform the user. Be mindful of the overall length of the conversation.
At any step, if it seems like the user is trying to leave, attempt to go through the exiting procedure.
"""
)
# These can only be one of the following values: "INIT_LEVEL" (default value), "CONTENT", or "EXIT".
# + Update to "EXIT" whenever the user is trying to exit the session.
# + Update to "CONTENT" whenever the user has selected a level and mode, and you are ready to generate content.

In [17]:
CONVERSATION_PROMPTS = {
    "WELCOME": """
You are now initializing the session. Say hello and welcome in French and introduce yourself in an enthusiastic way. 
Instruct the user to type `quit` to quit if they want. 
Ask the user to describe their current French level and desired learning mode.
""",
    "INIT_LEVEL": """
TASK: Understand the user's response regarding their CEFR level and update the user's level.
INSTRUCTIONS:
- You will receive the user's response when they are asked to describe their French level. It might be a simple answer like "A1" or "B2", a more complex one like "I am a beginner" or "I can read and write in French", or their confirmation regarding your previous conversations.
FEW SHOTS EXAMPLES:
+ "I am a beginner" -> Update user level to A1
+ "I can read and write fluently in French and I prefer to study actively" -> Update user level to B2 and mode to active
+ "I prefer to study passively" -> Update user mode to passive and ask the user to confirm their level
- You will then need to determine the user's CEFR level from their response. If uncertain, ask the user if what you think is correct.
- If the detected CEFR level and/or the study mode is confirmed, update each/both accordingly, otherwise, ask the user to clarify their level.
- You may repeat the question if the user refuses to or cannot provide a sufficient answer.
""",
    "CONTENT": """
TASK: Generate content based on the user's level, learning mode, and learned vocabulary.
INSTRUCTIONS:
- If the user has not set their learning mode yet, ask them to choose between passive and active learning.
- The content generated should be relevant to the user's level, study mode, and existing vocabulary.
- Based on the chat history, you may: 
+ For passive learning mode, generate content to teach new words, explain meaning and use in context. Avoid re-teaching previously learned vocabulary and focus on new words.
+ For active learning mode, generate a test or quiz based on the user's level and previously learned vocabulary. If the user has learned little to no vocabulary saved, you may then generate a quiz based on the user's level outside of that list.
+ Answer the user's questions and evaluate the user's answers during the lesson when applicable.
- After each successful interaction (new word learned, test passed, etc.), update the user's learned vocabulary. This should include all the words learned, not just the specified vocabulary. Do it quietly, without asking the user for confirmation.
- When asked, update the user's learning mode and CEFR level.
- If the user is trying to exit and has confirmed that is their intent, perform the tool call 'exit_session' and say nothing. 
""",
    "EXIT": """
TASK: The user has indicated they want to quit the conversation. Generate the updated vocab list for the user to copy into this file so that it can be saved for future sessions.
INSTRUCTIONS:
- 1: Generate a Python code that contains the learned vocabulary list, always named "history_learned_vocab".
- 2: The output should be nothing else but the code block, which can use Markdown syntax highlighting.
- 3: The code block should contain the learned vocabulary list, which is a list of strings.
- 4: The code block should be formatted in a way that the user can copy and paste it into their Python environment.
- 5: No other text should be included in the code block.
CODE BLOCK EXAMPLE:
```
# Copy and paste this code block into your Jupyter Notebook or Python environment to save your learned vocabulary.
history_learned_vocab = ["Bonjour", "Merci", "Au revoir"]
```
"""
}

UNKNOWN_STAGE_PROMPT = """
TASK: Process the user's previous input and determine the next step."""

In [18]:
def chatbot_with_tools(state: UserState) -> UserState:
    """The chatbot with tools. A simple wrapper around the model's own chat interface."""
    if debug:
        print(f"------ NODE: Chatbot")
    defaults = {"stage": "INIT_LEVEL", "learned_vocab": history_learned_vocab}
    if state["messages"]:
        current_stage = state["stage"]
        if current_stage not in CONVERSATION_PROMPTS:
            if debug:
                print(f"------ Unknown stage: {current_stage}, setting to UNKNOWN_STAGE.")
            current_stage = "UNKNOWN_STAGE"
        conversation_prompt = CONVERSATION_PROMPTS.get(current_stage)
        learned_vocab = state.get("learned_vocab", [])
        learned_vocab_prompt = f"User's learned vocabulary is: {', '.join(learned_vocab)}"
        if debug:
            print(f"------ Prompting AI for stage: {current_stage} with messages history, learned vocab, and prompt: {conversation_prompt}") #### Successfully printed
        new_output = llm.invoke([SYSINT, conversation_prompt, learned_vocab_prompt] + state["messages"])
        if debug:
            print("------ AI response (#1): ", new_output) #### Didn't print until after human's new input, although the new_output was generated before the human input
    else:
        conversation_prompt = CONVERSATION_PROMPTS["WELCOME"]
        if debug:
            print("------ Prompting AI to greet the user...")
        new_output = llm.invoke([SYSINT, conversation_prompt])
        if debug:
            print("------ AI greeting message (#2): ", new_output)

    # Return immediately if the output is not a tool call.
    if (hasattr(new_output, "content") and len(new_output.content) > 0):
       print(f"{bot_name}: ", new_output.content)

    return defaults | state | {"messages": [new_output]}

def human_node(state: UserState) -> UserState:
    """Display the last model message to the user, and receive the user's input."""
    if debug:
        print(f"------ NODE: Human node")
    # last_msg = state["messages"][-1]
    # print(f"{bot_name}: ", last_msg.content)

    response = input(f"{user_name}: ")
    if response in {"force quit"}:
        state["route_to"] = "end_node"
        return state
    
    print(f"{user_name}: ", response)
    return state | {"messages": [("user", response)]}


In [19]:
def maybe_route_to_tools(state: UserState) -> Literal["human_node", "process_tool_calls", "__end__"]:
    """Route between chat and tool nodes if a tool call is made."""
    if debug:
        print(f"------ ROUTING NODE: Maybe route to tools")
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if state.get("route_to", False) == "end_node" or state["stage"] == "EXIT":
        if debug:
            print(f"------ Routed to END")
        print(f"{bot_name}: Au revoir et à bientôt, paste the returned code block into your Python environment to save your learned vocabulary.")
        # print(msg.content) # may not need this
        # End node
        return END
    
    # When the chatbot returns tool_calls, route to the "tools" node.
    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        if debug:
            print(f"------ Routed to process_tool_calls")
        return "process_tool_calls"

    else:
        if debug:
            print(f"------ Routed to human_node")
        return "human_node"
    


def maybe_exit_human_node(state: UserState) -> Literal["chatbot_with_tools", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if debug:
        print(f"------ ROUTING NODE: Maybe exit human node")
    if state.get("route_to", False) == "end_node":
        if debug:
            print(f"------ Routed to END")
        return END
    else:
        if debug:
            print(f"------ Routed to chatbot_with_tools")
        return "chatbot_with_tools"

In [20]:
graph_builder = StateGraph(UserState)

graph_builder.add_node("chatbot_with_tools", chatbot_with_tools)
graph_builder.add_node("human_node", human_node)
graph_builder.add_node("process_tool_calls", process_tool_calls)

graph_builder.add_edge(START, "chatbot_with_tools")

graph_builder.add_edge("process_tool_calls", "chatbot_with_tools")

graph_builder.add_conditional_edges("chatbot_with_tools", maybe_route_to_tools)
graph_builder.add_conditional_edges("human_node", maybe_exit_human_node)

graph_builder = graph_builder.compile()

In [21]:
local_debug = False
if local_debug != None:
    debug = local_debug

from pprint import pprint

config = RunnableConfig(recursion_limit=recursion_limit)
state = graph_builder.invoke({"messages": []}, config=config)

if debug:
    pprint(state)

Frenchie:  Bonjour et bienvenue ! I am Frenchie, your personal French vocabulary learning assistant! I'm super excited to help you expand your French vocabulary.

If you want to quit at any time, just type `quit`.

To start, could you please tell me your current French level (e.g., A1, A2, B1, B2, C1, or C2) and whether you prefer a passive or active learning mode?
You:  B1 and active
Frenchie:  Parfait! I will set your level to B1 and your learning mode to active. Is that correct?
You:  yes
Frenchie:  Super! Now that you're in active learning mode at the B1 level, let's test your knowledge.

Here's your first question:

Translate the following sentence into French: "I am looking for my red apple."
You:  Je cherche ma pomme rouge
Frenchie:  That's very close! "Je cherche ma pomme rouge" is a correct translation.

Since you translated "pomme" and "rouge" correctly and they are already in your learned vocabulary, let's add "cherche" and "chercher" (the infinitive form) to your vocabulary

In [15]:
history_learned_vocab = ["bonjour", "au revoir", "excusez-moi", "bonsoir", "merci"]
